In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import glob
import os
import datetime as dt
from datetime import datetime

%matplotlib inline

In [10]:
tickers = ['FB', 'AA', 'AAPL']
stocks = download_tickers(tickers)

['FB', 'AAPL', 'AA']
already downloaded FB
already downloaded AA
already downloaded AAPL


In [11]:
stocks

Stock ticker          FB                                                  \
Data type           Open        High         Low       Close   Adj Close   
1962-01-02           NaN         NaN         NaN         NaN         NaN   
1962-01-03           NaN         NaN         NaN         NaN         NaN   
1962-01-04           NaN         NaN         NaN         NaN         NaN   
1962-01-05           NaN         NaN         NaN         NaN         NaN   
1962-01-08           NaN         NaN         NaN         NaN         NaN   
...                  ...         ...         ...         ...         ...   
2020-08-31    293.950012  296.880005  291.549988  293.200012  293.200012   
2020-09-01    294.709991  301.489990  292.709991  295.440002  295.440002   
2020-09-02    298.880005  303.600006  293.049988  302.500000  302.500000   
2020-09-03    295.989990  297.600006  283.630005  291.119995  291.119995   
2020-09-04    287.250000  289.000000  271.140015  282.730011  282.730011   

Stock ticker                     AA                                   \
Data type         Volume       Open       High        Low      Close   
1962-01-02           NaN   6.532155   6.556185   6.532155   6.532155   
1962-01-03           NaN   6.532155   6.632280   6.524145   6.632280   
1962-01-04           NaN   6.632280   6.664320   6.632280   6.632280   
1962-01-05           NaN   6.632280   6.656310   6.616260   6.624270   
1962-01-08           NaN   6.608250   6.608250   6.339915   6.408000   
...                  ...        ...        ...        ...        ...   
2020-08-31    17345100.0  15.230000  15.230000  14.620000  14.620000   
2020-09-01    17320900.0  14.640000  15.180000  14.240000  15.090000   
2020-09-02    24341400.0  14.730000  14.870000  14.410000  14.670000   
2020-09-03    32198700.0  14.530000  14.780000  14.010000  14.230000   
2020-09-04    29809863.0  14.520000  14.670000  13.900000  14.400000   

Stock ticker                            AAPL                          \
Data type     Adj Close   Volume        Open        High         Low   
1962-01-02     1.536658    55900         NaN         NaN         NaN   
1962-01-03     1.560212    74500         NaN         NaN         NaN   
1962-01-04     1.560212    80500         NaN         NaN         NaN   
1962-01-05     1.558326    70500         NaN         NaN         NaN   
1962-01-08     1.507450    93800         NaN         NaN         NaN   
...                 ...      ...         ...         ...         ...   
2020-08-31    14.620000  4033200  127.580002  131.000000  126.000000   
2020-09-01    15.090000  4123200  132.759995  134.800003  130.529999   
2020-09-02    14.670000  5691200  137.589996  137.979996  127.000000   
2020-09-03    14.230000  3923300  126.910004  128.839996  120.500000   
2020-09-04    14.400000  3902969  120.070000  123.699997  110.889999   

Stock ticker                                       
Data type          Close   Adj Close       Volume  
1962-01-02           NaN         NaN          NaN  
1962-01-03           NaN         NaN          NaN  
1962-01-04           NaN         NaN          NaN  
1962-01-05           NaN         NaN          NaN  
1962-01-08           NaN         NaN          NaN  
...                  ...         ...          ...  
2020-08-31    129.039993  129.039993  225702700.0  
2020-09-01    134.179993  134.179993  152470100.0  
2020-09-02    131.399994  131.399994  200119000.0  
2020-09-03    120.879997  120.879997  254723200.0  
2020-09-04    120.959999  120.959999  332607163.0  

[14772 rows x 18 columns]

In [4]:
def download_tickers(tickers):
    # Fetch downloaded ticker from raw_data
    downloaded_tickers = [x[0:-5] if '.json' in x else '' for x in os.listdir('raw_data/')]
    downloaded_tickers.remove('')
    print(downloaded_tickers)
    # Check if tickers already are downloaded
    tickers_to_download = []
    for ticker in tickers:
        if ticker in downloaded_tickers:
            # If already downloaded, check if they need updates
            if update_ticker(ticker):
                tickers_to_download.append(ticker)
        else:
            tickers_to_download.append(ticker)

    download_dump(tickers_to_download)
    
    # Read tickers from json and return
    return load_stocks(tickers)

In [5]:
def load_stocks(tickers):
    cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    index = pd.MultiIndex.from_product([tickers, cols], names=['Stock ticker', 'Data type'])
    data = pd.concat([pd.read_json('raw_data/' + x + '.json', ) for x in tickers], axis=1, sort=True)
    data.columns = index
    data.index = pd.to_datetime(data.index, unit='ms')
    return data

In [6]:
def update_ticker(ticker):
    last_bday = last_weekday()
    newest_date = pd.read_json('raw_data/' + ticker + '.json').index[-1]
    newest_date = dt.datetime.fromtimestamp(newest_date/1000).date()
    if newest_date != last_bday:
        return True 
    else:
        print('already downloaded', ticker)
        return False

In [8]:
def download_dump(tickers):
    if tickers != []:
        data = yf.download(tickers, start='1900-01-01', group_by='tickers')
        if len(tickers) > 1:
            for ticker in tickers:
                    data[ticker].to_json('raw_data/' + ticker + '.json', date_format='epoch', date_unit='ms')
        else:
            data.to_json('raw_data/' + tickers[0] + '.json', date_format='epoch', date_unit='ms')
        print('Downloaded:', tickers)

In [9]:
def last_weekday():
    todays_day = dt.date.today().day
    if todays_day in [5, 6]:
        return dt.date.today() - dt.timedelta(days=todays_day-4)
    else:
        return dt.date.today()
